In [ ]:
# Data download

!gdown --id '1VQry5JMRcuZ_BStIX8-FB8n4zFuDNUjk'

Downloading...
From: https://drive.google.com/uc?id=1VQry5JMRcuZ_BStIX8-FB8n4zFuDNUjk
To: /content/BITCOIN_HORA_A_HORA_2019_2022_BEST_DATA_2.csv
100% 1.95M/1.95M [00:00<00:00, 114MB/s]


# TRATAMENTO E VALIDAÇÃO DE DADOS

In [ ]:
# Imports

import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# Formatação para suprimir notação científica na hora
# de mostrar os tensores do NumPy na tela.

np.set_printoptions(formatter={'float':lambda x: '%+01.2f ' % x})

In [ ]:
# MAIN SETUP

INPUT_SIZE = 11

In [ ]:
# Open File

f = open('BITCOIN_HORA_A_HORA_2019_2022_BEST_DATA_2.csv','r')
f.seek(0)
lines = f.readlines()[2:-1] # Read and remove top and bottom and first data, 'cause of diff before

In [ ]:
# Array temporário que receberá os dados (PRICES e PREV) tratados:
X_TEMP = np.zeros((len(lines),1))
Y_TEMP = np.zeros((len(lines),2))
print(len(lines))
print(len(X_TEMP))

21999
21999


In [ ]:
for i, line in enumerate(lines):

  # Lendo linha menos o último caractere que é o \n
  s = line[:-1]
  # print(s)

  # Splitando na vírgula (CSV) e atribuido valores a variáveis
  _, open, close, low, high, volumefrom, volumeto, diff_beff, prev, diff_next = s.split(',')

  # Cast para float
  # data_cast = float(float(close) * float(volumefrom) * float(diff_beff))
  # data_cast = float(diff_beff)

  open = float(open)
  close = float(close)
  low = float(low)
  high = float(high)
  volumefrom = float(volumefrom)
  volumeto = float(volumeto)
  diff_beff = float(diff_beff)
  diff_next = float(diff_next)


  data_cast = ((open-close)*(high-low))/volumeto

  
  # Salvando valores da array do numpy
  X_TEMP[i] = np.array([data_cast])

  # ONE-HOT

  if prev == 'UP':
    Y_TEMP[i] = np.array([1,0])
  elif prev == 'DOWN':
    Y_TEMP[i] = np.array([0,1])



In [ ]:
# # Validar ONE-HOT
# for d in range(len(X_TEMP)):
#   print(X_TEMP[d],Y_TEMP[d])

In [ ]:
# Array com 10 preços em sequencia
# Array para one-hot (UP/DOWN)

X = np.zeros((len(lines)-INPUT_SIZE ,INPUT_SIZE))
Y = np.zeros((len(lines)-INPUT_SIZE ,2))

In [ ]:
# Create horizontal array with 10 last prices

for i in range(len(X_TEMP)-INPUT_SIZE):

  for j in range(INPUT_SIZE):
    X[i][j] = float(X_TEMP[i+j])


In [ ]:
# Andamos 10 casas para frente em Y_T (9 por considerar o 0)

Y = Y_TEMP[(INPUT_SIZE-1):-1]

# Verificando quantidade de dados:

print('X LEN = ', len(X))
print('Y LEN = ', len(Y))


X LEN =  21988
Y LEN =  21988


In [ ]:
# # ONE-HOT Final Validation

# # [1,0] = NEXT UP
# # [0,1] = NEXT DOWN

# for d in range(len(X)):
#   print(X[d,(INPUT_SIZE-1)],Y[d])

In [ ]:
# Separando dados em treinamento e validação


# Random data
# indexes = list(range(len(X)))               
# np.random.shuffle(indexes)

TRAIN = int(len(X)*0.95) # 90% of data for training


X_TRAIN = np.zeros((TRAIN,INPUT_SIZE))
Y_TRAIN = np.zeros((TRAIN,2))

X_VALIDATE = np.zeros((len(X)-TRAIN,INPUT_SIZE))
Y_VALIDATE = np.zeros((len(X)-TRAIN,2))

# for i in range(0,TRAIN):
#   X_TRAIN[i,:] = X[indexes[i],:]
#   Y_TRAIN[i,:] = Y[indexes[i],:]
# for i in range(0,len(X)-TRAIN):
#   X_VALIDATE[i,:] = X[indexes[TRAIN+i],:]
#   Y_VALIDATE[i,:] = Y[indexes[TRAIN+i],:]

for i in range(0,TRAIN):
  X_TRAIN[i,:] = X[i,:]
  Y_TRAIN[i,:] = Y[i,:]
for i in range(0,len(X)-TRAIN):
  X_VALIDATE[i,:] = X[TRAIN+i,:]
  Y_VALIDATE[i,:] = Y[TRAIN+i,:]


In [ ]:
# for i in range(len(X_VALIDATE)):
#   print(X_VALIDATE[i],Y_VALIDATE[i])

# REDE NEURAL


In [ ]:
# Imports

import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

from keras.callbacks import ModelCheckpoint

# Formatação para suprimir notação científica na hora
# de mostrar os tensores do NumPy na tela.

np.set_printoptions(formatter={'float':lambda x: '%+01.2f ' % x})

print('INPUT_SIZE = ', INPUT_SIZE)

INPUT_SIZE =  11


In [ ]:
# Definindo arquitetura da rede neural

model = keras.Sequential(
    [
     keras.Input(shape=(INPUT_SIZE)),

     layers.Dense(32, activation='relu'),

    #  layers.Dropout(0.2),

     layers.Dense(32, activation='relu'),

    #  layers.Dense(64, activation='relu'),

     layers.Dense(27, activation='relu'),


    #  layers.Dropout(0.2),

     layers.Dense(2, activation="softmax")

    ]
);

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 32)                384       
                                                                 
 dense_5 (Dense)             (None, 32)                1056      
                                                                 
 dense_6 (Dense)             (None, 27)                891       
                                                                 
 dense_7 (Dense)             (None, 2)                 56        
                                                                 
Total params: 2,387
Trainable params: 2,387
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compilamos o Modelo

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']);

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']);


# model.compile(loss='crossentropy', optimizer='adam', metrics=['accuracy']);

              

In [ ]:
# Treinamento

# filepath = 'my_best_model.epoch{epoch:02d}-loss{val_loss:.2f}.hdf5'

# filepath = 'BEST_MODEL_ACC-{accuracy:.2f}.hdf5'

filepath = 'BEST_MODEL_NEW.hdf5'

monitor = 'accuracy'

checkpoint = ModelCheckpoint(filepath = filepath, 
                             monitor = monitor,
                             verbose = 1, 
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint]

# best = keras.models.load_model('/content/my_best_model.epoch01-loss1.99.hdf5')

# model.fit(X_TRAIN, Y_TRAIN, batch_size=len(X_TRAIN), epochs=10, validation_split=0.2,callbacks = callbacks)

model.fit(X_TRAIN, Y_TRAIN, batch_size=len(X_TRAIN), epochs=1001, callbacks = callbacks)


Epoch 1/1001
1/1 [==============================] - ETA: 0s - loss: 0.6930 - accuracy: 0.5076
Epoch 1: accuracy improved from -inf to 0.50761, saving model to BEST_MODEL_NEW.hdf5
1/1 [==============================] - 0s 49ms/step - loss: 0.6930 - accuracy: 0.5076
Epoch 2/1001
1/1 [==============================] - ETA: 0s - loss: 0.6930 - accuracy: 0.5076
Epoch 2: accuracy did not improve from 0.50761
1/1 [==============================] - 0s 14ms/step - loss: 0.6930 - accuracy: 0.5076
Epoch 3/1001
1/1 [==============================] - ETA: 0s - loss: 0.6930 - accuracy: 0.5076
Epoch 3: accuracy did not improve from 0.50761
1/1 [==============================] - 0s 17ms/step - loss: 0.6930 - accuracy: 0.5076
Epoch 4/1001
1/1 [==============================] - ETA: 0s - loss: 0.6930 - accuracy: 0.5076
Epoch 4: accuracy did not improve from 0.50761
1/1 [==============================] - 0s 17ms/step - loss: 0.6930 - accuracy: 0.5076
Epoch 5/1001
1/1 [==============================] - ET

In [ ]:
# Load and Evaluate the Best saved Model

best = keras.models.load_model('BEST_MODEL_NEW.hdf5')

loss, acc = best.evaluate(X_TRAIN, Y_TRAIN, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))


653/653 - 2s - loss: 0.6793 - accuracy: 0.5638 - 2s/epoch - 3ms/step
Restored model, accuracy: 56.38%


In [ ]:
# Verify Results

# best = keras.models.load_model(filepath)


# Na esquerda a saída desejada,
# na direita a calculada pela rede

# FAST_TEST = 1000
# FAST_TEST = 1090
FAST_TEST = 0


right = 0
wrong = 0

for i in range(X_VALIDATE.shape[0] - FAST_TEST):
  x = X_VALIDATE[i:i+1,:]
  y_hat = Y_VALIDATE[i:i+1,:]
  y = best.predict(x)

  # print(y_hat,y)
  # print(keras.metrics.categorical_accuracy(y_hat,y))

  if y[0][0] > y[0][1]:
    res = 1 # UP
  else:
    res = 0 # DOWN

  if y_hat[0][0] == res:
    right = right+1
    print(y_hat,y," - YES")

  elif y_hat[0][0] != res:
    wrong = wrong+1
    print(y_hat,y," - NO")

ac = (right)/(right+wrong)
print('YES = ', right)
print('NO = ', wrong)
print('Total = ', (right+wrong))
print('Acurracy = ', ac)


[[+0.00  +1.00 ]] [[+0.40  +0.60 ]]  - YES
[[+1.00  +0.00 ]] [[+0.46  +0.54 ]]  - NO
[[+1.00  +0.00 ]] [[+0.39  +0.61 ]]  - NO
[[+0.00  +1.00 ]] [[+0.38  +0.62 ]]  - YES
[[+0.00  +1.00 ]] [[+0.54  +0.46 ]]  - NO
[[+1.00  +0.00 ]] [[+0.61  +0.39 ]]  - YES
[[+0.00  +1.00 ]] [[+0.58  +0.42 ]]  - NO
[[+1.00  +0.00 ]] [[+0.56  +0.44 ]]  - YES
[[+0.00  +1.00 ]] [[+0.54  +0.46 ]]  - NO
[[+0.00  +1.00 ]] [[+0.56  +0.44 ]]  - NO
[[+0.00  +1.00 ]] [[+0.66  +0.34 ]]  - NO
[[+0.00  +1.00 ]] [[+0.59  +0.41 ]]  - NO
[[+1.00  +0.00 ]] [[+0.62  +0.38 ]]  - YES
[[+0.00  +1.00 ]] [[+0.57  +0.43 ]]  - NO
[[+1.00  +0.00 ]] [[+0.62  +0.38 ]]  - YES
[[+0.00  +1.00 ]] [[+0.53  +0.47 ]]  - NO
[[+1.00  +0.00 ]] [[+0.57  +0.43 ]]  - YES
[[+1.00  +0.00 ]] [[+0.43  +0.57 ]]  - NO
[[+0.00  +1.00 ]] [[+0.57  +0.43 ]]  - NO
[[+1.00  +0.00 ]] [[+0.52  +0.48 ]]  - YES
[[+1.00  +0.00 ]] [[+0.61  +0.39 ]]  - YES
[[+0.00  +1.00 ]] [[+0.52  +0.48 ]]  - NO
[[+0.00  +1.00 ]] [[+0.37  +0.63 ]]  - YES
[[+1.00  +0.00 ]] [[+0.5